In [92]:
import json
import pymorphy2
import pandas as pd

In [289]:
data = json.load(open('weather_data.json'))

In [290]:
data = data[0]
headers = data['dd']
del data['dd']

In [291]:
data['Средняя температура']=list(map(lambda x: int(x),np.round((np.array(data['Минимальная температура'])+np.array(data['Максимальная температура']))/2)))

In [292]:
data

{'Максимальная температура': [-7, -5, -3, -4, -5, -7, -10, -11, -4, -5],
 'Минимальная температура': [-11, -8, -5, -6, -7, -11, -16, -18, -11, -11],
 'Скорость ветра': [6, 8, 8, 8, 7, 7, 6, 13, 15, 12],
 'Средняя температура': [-9, -6, -4, -5, -6, -9, -13, -14, -8, -8],
 'Уровень осадков': [0.5, 0.8, 2.5, 2.0, 4.0, 0.2, 0.0, 2.0, 7.4, 0.6]}

In [293]:
headers

['Чт 15.02',
 'Пт 16.02',
 'Сб 17.02',
 'Вс 18.02',
 'Пн 19.02',
 'Вт 20.02',
 'Ср 21.02',
 'Чт 22.02',
 'Пт 23.02',
 'Сб 24.02']

In [294]:
df = pd.DataFrame.from_dict(data).transpose()
df.columns = headers

In [295]:
df = df.reindex(['Максимальная температура','Минимальная температура','Средняя температура','Скорость ветра','Уровень осадков'])

In [296]:
df

,Чт 15.02,Пт 16.02,Сб 17.02,Вс 18.02,Пн 19.02,Вт 20.02,Ср 21.02,Чт 22.02,Пт 23.02,Сб 24.02
Максимальная температура,-7.0,-5.0,-3.0,-4.0,-5.0,-7.0,-10.0,-11.0,-4.0,-5.0
Минимальная температура,-11.0,-8.0,-5.0,-6.0,-7.0,-11.0,-16.0,-18.0,-11.0,-11.0
Средняя температура,-9.0,-6.0,-4.0,-5.0,-6.0,-9.0,-13.0,-14.0,-8.0,-8.0
Скорость ветра,6.0,8.0,8.0,8.0,7.0,7.0,6.0,13.0,15.0,12.0
Уровень осадков,0.5,0.8,2.5,2.0,4.0,0.2,0.0,2.0,7.4,0.6


In [146]:
days_dict = {'Пн':'понедельник','Вт':'вторник','Ср':'среда','Чт':'четверг','Пт':'пятница','Сб':'суббота','Вс':'воскресенье'}

In [37]:
def morph_analyze(word):
    morph = pymorphy2.MorphAnalyzer()
    return morph.parse(word)[0]

In [236]:
def prep_day_agree(Prep,Day):
    if Prep.lower() == 'в':
        if Day == 'вторник':
            return morph_analyze(Prep).inflect({'Vpre'}).word +' '+  morph_analyze(Day).inflect({'accs'}).word
        else:
             return Prep +' '+  morph_analyze(Day).inflect({'accs'}).word   
    if Prep.lower() == 'с':
        if Day == 'вторник' or Day == 'среда':
            return morph_analyze(Prep).inflect({'Vpre'}).word +' '+  morph_analyze(Day).inflect({'ablt'}).word
        else:
             return Prep +' '+  morph_analyze(Day).inflect({'ablt'}).word   

In [237]:
def gen_template_temp(D1,D2,City_Name):
    D1_day = days_dict[D1.split()[0]]
    D1_date = D1.split()[1]
    D2_day = days_dict[D2.split()[0]]
    D2_date = D2.split()[1]
    
    temp_trend = ''
    City_Name_agreed = morph_analyze(City_Name).inflect({'loct'}).word.title()
    
    D1_med_temp = int(df[D1]['Средняя температура'])
    D2_med_temp = int(df[D2]['Средняя температура'])
    
    if D1_med_temp > D2_med_temp:
        temp_trend = "потеплеет"
    elif D1_med_temp < D2_med_temp:
        temp_trend = "похолодает"
    else:
        #import pdb; pdb.set_trace()
        return prep_day_agree('В',D1_day)+' '+D1_date+' в '+City_Name_agreed+' температура не изменится по сравнению '+ prep_day_agree('с',D2_day)+' '+D2_date+' ('+str(D1_med_temp)+' '+morph_analyze('градус').make_agree_with_number(abs(D1_med_temp)).word+' ).'
    
    return prep_day_agree('В',D1_day).title()+' '+D1_date+' в '+City_Name_agreed+' '+temp_trend+' '+'на '+str(abs(D1_med_temp-D2_med_temp))+' '+morph_analyze('градус').make_agree_with_number(abs(D1_med_temp-D2_med_temp)).word+' по сравнению'+' '+prep_day_agree('с',D2_day)+' '+D2_date+'.'

In [238]:
for day in df.columns:
    print(gen_template_temp(df.columns[-1],day,'Москва'))

В Субботу 24.02 в Москве потеплеет на 1 градус по сравнению с четвергом 15.02.
В Субботу 24.02 в Москве похолодает на 2 градуса по сравнению с пятницей 16.02.
В Субботу 24.02 в Москве похолодает на 4 градуса по сравнению с субботой 17.02.
В Субботу 24.02 в Москве похолодает на 3 градуса по сравнению с воскресеньем 18.02.
В Субботу 24.02 в Москве похолодает на 2 градуса по сравнению с понедельником 19.02.
В Субботу 24.02 в Москве потеплеет на 1 градус по сравнению со вторником 20.02.
В Субботу 24.02 в Москве потеплеет на 5 градусов по сравнению со средой 21.02.
В Субботу 24.02 в Москве потеплеет на 6 градусов по сравнению с четвергом 22.02.
В субботу 24.02 в Москве температура не изменится по сравнению с пятницей 23.02 (-8 градусов ).
В субботу 24.02 в Москве температура не изменится по сравнению с субботой 24.02 (-8 градусов ).


In [297]:
df

,Чт 15.02,Пт 16.02,Сб 17.02,Вс 18.02,Пн 19.02,Вт 20.02,Ср 21.02,Чт 22.02,Пт 23.02,Сб 24.02
Максимальная температура,-7.0,-5.0,-3.0,-4.0,-5.0,-7.0,-10.0,-11.0,-4.0,-5.0
Минимальная температура,-11.0,-8.0,-5.0,-6.0,-7.0,-11.0,-16.0,-18.0,-11.0,-11.0
Средняя температура,-9.0,-6.0,-4.0,-5.0,-6.0,-9.0,-13.0,-14.0,-8.0,-8.0
Скорость ветра,6.0,8.0,8.0,8.0,7.0,7.0,6.0,13.0,15.0,12.0
Уровень осадков,0.5,0.8,2.5,2.0,4.0,0.2,0.0,2.0,7.4,0.6


In [271]:
def num_agree_word(num,word):
    if abs(num) == 1:
        return morph_analyze(word).inflect({'accs'}).word
    else:
        return morph_analyze(word).make_agree_with_number(abs(num)).word

In [299]:
def gen_template_wind(D1,D2):
    D1_day = days_dict[D1.split()[0]]
    D1_date = D1.split()[1]
    D2_day = days_dict[D2.split()[0]]
    D2_date = D2.split()[1]
    
    D1_wind = int(df[D1]['Скорость ветра'])
    D2_wind = int(df[D2]['Скорость ветра'])
    
    return 'Скорость ветра изменится на '+str(D1_wind-D2_wind)+' '+ num_agree_word(D1_wind-D2_wind,'метр')+' в секунду '+' '+prep_day_agree('в',D1_day)+' '+D1_date+' по сравнению'+' '+prep_day_agree('с',D2_day)+' '+D2_date+'.'

In [300]:
for day in df.columns:
    print(gen_template_wind(df.columns[-1],day))

Скорость ветра изменится на 6 метров в секунду  в субботу 24.02 по сравнению с четвергом 15.02.
Скорость ветра изменится на 4 метра в секунду  в субботу 24.02 по сравнению с пятницей 16.02.
Скорость ветра изменится на 4 метра в секунду  в субботу 24.02 по сравнению с субботой 17.02.
Скорость ветра изменится на 4 метра в секунду  в субботу 24.02 по сравнению с воскресеньем 18.02.
Скорость ветра изменится на 5 метров в секунду  в субботу 24.02 по сравнению с понедельником 19.02.
Скорость ветра изменится на 5 метров в секунду  в субботу 24.02 по сравнению со вторником 20.02.
Скорость ветра изменится на 6 метров в секунду  в субботу 24.02 по сравнению со средой 21.02.
Скорость ветра изменится на -1 метр в секунду  в субботу 24.02 по сравнению с четвергом 22.02.
Скорость ветра изменится на -3 метра в секунду  в субботу 24.02 по сравнению с пятницей 23.02.
Скорость ветра изменится на 0 метров в секунду  в субботу 24.02 по сравнению с субботой 24.02.


In [298]:
df

,Чт 15.02,Пт 16.02,Сб 17.02,Вс 18.02,Пн 19.02,Вт 20.02,Ср 21.02,Чт 22.02,Пт 23.02,Сб 24.02
Максимальная температура,-7.0,-5.0,-3.0,-4.0,-5.0,-7.0,-10.0,-11.0,-4.0,-5.0
Минимальная температура,-11.0,-8.0,-5.0,-6.0,-7.0,-11.0,-16.0,-18.0,-11.0,-11.0
Средняя температура,-9.0,-6.0,-4.0,-5.0,-6.0,-9.0,-13.0,-14.0,-8.0,-8.0
Скорость ветра,6.0,8.0,8.0,8.0,7.0,7.0,6.0,13.0,15.0,12.0
Уровень осадков,0.5,0.8,2.5,2.0,4.0,0.2,0.0,2.0,7.4,0.6


In [285]:
df.columns.get_loc('Пт 16.02')

1

In [288]:
df.columns.shape[0]

10

Уровень осадков повысится / понизится на X единиц за Y дней.

In [318]:
def num_agree_word_prec(num,word):
    if num.is_integer():
        return morph_analyze(word).make_agree_with_number(int(num)).word
    else:
        return morph_analyze(word).inflect({'gent'}).word # при смешанном числе существительным управляет дробь

In [319]:
def gen_template_prec(D,Period):
    offset = df.columns.get_loc(D)+Period
    if (offset) >= df.columns.shape[0]:
        return 'Недостаточно данных в таблице'
    
    prec_diff = df[df.columns[offset]]['Уровень осадков']-df[D]['Уровень осадков']
    prec_trend = ''
    
    if prec_diff > 0:
        prec_trend = 'повысится'
    elif prec_diff < 0:
        prec_trend = 'понизится'
    else:
        return 'Уровень осадков не изменится за '+str(Period)+' '+morph_analyze('день').make_agree_with_number(Period).word+' относительно '+D+'.'
    return 'Уровень осадков '+prec_trend+' на '+str(abs(prec_diff))+' '+num_agree_word_prec(abs(prec_diff),'миллиметр')+' за '+str(Period)+' '+morph_analyze('день').make_agree_with_number(Period).word+' относительно '+D+'.'

In [324]:
for i in range(1,10):
    print(gen_template_prec(df.columns[0],i))

Уровень осадков повысится на 0.3 миллиметра за 1 день относительно Чт 15.02.
Уровень осадков повысится на 2.0 миллиметра за 2 дня относительно Чт 15.02.
Уровень осадков повысится на 1.5 миллиметра за 3 дня относительно Чт 15.02.
Уровень осадков повысится на 3.5 миллиметра за 4 дня относительно Чт 15.02.
Уровень осадков понизится на 0.3 миллиметра за 5 дней относительно Чт 15.02.
Уровень осадков понизится на 0.5 миллиметра за 6 дней относительно Чт 15.02.
Уровень осадков повысится на 1.5 миллиметра за 7 дней относительно Чт 15.02.
Уровень осадков повысится на 6.9 миллиметра за 8 дней относительно Чт 15.02.
Уровень осадков повысится на 0.1 миллиметра за 9 дней относительно Чт 15.02.
